<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install --upgrade implicit

In [54]:
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta


# Loading DataFrames



In [4]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-4-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [5]:
len(df)

2220299

In [32]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped,purchase
7402,4428029,23.11.22,NaN,2022FM12,44504136,THINK ABOUT IT GMBH,Business Channel,,CF58186,IPHONE 14 PRO 256GB,SPACE BLACK,1402,Mobility,Smart Phones,Printers & Peripherals,"1.118,04",1.0,1
7902,4435364,24.11.22,NaN,2022FM11,44423496,KOSATEC COMPUTER GMBH,Mid Market,,406A863,27BN55U 27IN 68.58CM IPS,3840 X 2160 16 : 9 HDMI,320,Display,Lcd Monitor,Printers & Peripherals,"8.432,00",34.0,1


# Data Preprocessing

In [6]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)

In [33]:
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)

In [34]:
# lines with zero shipment qty
len(df[df["Qty Shipped"] == 0])

51412

In [36]:
df[df["Qty Shipped"] == 0].head(50)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped,purchase
77868,4425643,30.06.23,30.06.2023 16:40:56,2023FM06,44442174,COMPUTERUNIVERSE GMBH,Consumer Channel,CUU,CF70217,ZBOOK FURY 16 G9 CI7-12800HX,16.0WUXGA 1TB 2X16GB W11P A2000,11,Computer Systems,Portable/Notebook Computers,System,"0,00",0.0,1
77993,4425414,30.06.23,30.06.2023 16:03:19,2023FM06,44431805,MR-DATENTECHNIK,Business Channel,MR,CC93790,IPHONE 13 MINI 128GB MIDNIGHT,,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0,1
77998,4425406,30.06.23,30.06.2023 16:01:28,2023FM06,44431805,MR-DATENTECHNIK,Business Channel,MR,CC93790,IPHONE 13 MINI 128GB MIDNIGHT,,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0,1
78016,4425353,30.06.23,30.06.2023 15:53:59,2023FM06,44442314,KOMSA AG,Mid Market,,CA07583,OTTERBOX PREMIUM CABLE USB AC,2M BLACK,1206,Cables,Usb Cables & Adapters,Printers & Peripherals,"0,00",0.0,1
78017,4425353,30.06.23,30.06.2023 15:53:59,2023FM06,44442314,KOMSA AG,Mid Market,,CA07799,OTTERBOX REACT IPHONE 12 MINI,CLEAR,1421,Mobility,Housings / Covers,Printers & Peripherals,"0,00",0.0,1
78020,4425353,30.06.23,30.06.2023 15:53:59,2023FM06,44442314,KOMSA AG,Mid Market,,CA86358,OTTERBOX REACT SAMSUNG GALAXY,A52/A52 5G/A52S 5G - BLACK,1421,Mobility,Housings / Covers,Printers & Peripherals,"0,00",0.0,1
78024,4425353,30.06.23,30.06.2023 15:53:59,2023FM06,44442314,KOMSA AG,Mid Market,,CB74542,REACT IPHONE 13 CLEAR,CLEAR,1421,Mobility,Housings / Covers,Printers & Peripherals,"0,00",0.0,1
78028,4425353,30.06.23,30.06.2023 15:53:59,2023FM06,44442314,KOMSA AG,Mid Market,,CD54658,OTTERBOX DEFENDER APPLE IPAD,MINI 6TH GEN - BLACK,16,Computer Systems,Tablet Pc Accs,System,"0,00",0.0,1
78029,4425353,30.06.23,30.06.2023 15:53:59,2023FM06,44442314,KOMSA AG,Mid Market,,CD54722,DEFENDER SAMSUNG GALAXY S22,BLACK,1421,Mobility,Housings / Covers,Printers & Peripherals,"0,00",0.0,1
78031,4425353,30.06.23,30.06.2023 15:53:59,2023FM06,44442314,KOMSA AG,Mid Market,,CD94273,POWER BANK 15K MAH USB A AND C,18W USB-PD + WIRELESS 10W BLACK,1413,Mobility,Power Options,Printers & Peripherals,"0,00",0.0,1


In [37]:
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"]> 0]

In [38]:
# dropping backlog invoices
df = df[df["Entry Date"] >= '2022-10-01 00:00:00']

In [39]:
len(df)

685862

In [40]:
# implementing a purchase frequency counter helper
df.loc[:, "purchase"] = 1

## User and sku unqiue list

In [42]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [47]:
df_1 = df[["bcn_id","sku_id","Qty Shipped", "purchase"]]

In [48]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 685862 entries, 7402 to 2220294
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   bcn_id       685862 non-null  int16  
 1   sku_id       685862 non-null  int32  
 2   Qty Shipped  685862 non-null  float64
 3   purchase     685862 non-null  int64  
dtypes: float64(1), int16(1), int32(1), int64(1)
memory usage: 19.6 MB


In [49]:
df_1.head(3)

,bcn_id,sku_id,Qty Shipped,purchase
7402,4820,33457,1.0,1
7902,1144,8567,34.0,1
7903,6140,23686,3.0,1


In [50]:
grouped_df = df.groupby(["bcn_id", "sku_id"]).agg({
    "Qty Shipped": "sum",
    "purchase": "sum"
}).reset_index()

In [52]:
grouped_df.sort_values(by="bcn_id",ascending=True)

,bcn_id,sku_id,Qty Shipped,purchase
0,0,2276,1.0,1
28,0,24564,1.0,1
29,0,27140,5.0,2
30,0,27167,2.0,1
31,0,27340,3.0,1
...,...,...,...,...
284193,9885,1816,6.0,2
284194,9885,28315,6.0,2
284195,9886,26057,3.0,1
284196,9887,28303,2.0,1


## Create sparse user x item and item x user matrices

In [53]:
sparse_item_user = sparse.csr_matrix((grouped_df['purchase'].astype(float), (grouped_df['sku_id'], grouped_df['bcn_id'])))
sparse_user_item = sparse.csr_matrix((grouped_df['purchase'].astype(float), (grouped_df['bcn_id'], grouped_df['sku_id'])))

# **Als Model**

In [23]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


## LOOCV

with K=1 so 1 item per user gets dropped out

In [58]:
x = leave_k_out_split(sparse_item_user, K=1)
sparse_item_user_train = x[0]
sparse_item_user_test = x[1]

In [56]:
y = leave_k_out_split(sparse_user_item, K=1)
sparse_user_item_train = y[0]
sparse_user_item_test = y[1]

## Model param tuning

In [26]:
factors = 128
regularization = 0.01
iterations= 10
calculate_training_loss=True

In [27]:
model = implicit.als.AlternatingLeastSquares(factors=factors, regularization=regularization, iterations=iterations)

In [28]:
alpha_val = 40
data_conf = (sparse_user_item_train * alpha_val).astype('float')

model.fit(data_conf)

  0%|          | 0/10 [00:00<?, ?it/s]

## Best Model Performance

In [29]:
precision_als = precision_at_k(model, sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
precision_als

  0%|          | 0/5948 [00:00<?, ?it/s]

0.18207800941492938

In [30]:
mean_precision_k = mean_average_precision_at_k(model, sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
mean_precision_k

  0%|          | 0/5948 [00:00<?, ?it/s]

0.09305075202015343

In [31]:
ndcg = ndcg_at_k(model, sparse_user_item_train, sparse_user_item_test, show_progress=True, K=10, num_threads=10)
ndcg

  0%|          | 0/5948 [00:00<?, ?it/s]

0.11390330284538876

# Model Recommendations

In [32]:
 # Assuming you want recommendations for user with ID 10
user_id = 7010
# Get the row of the user from the sparse_user_item matrix
user_row = sparse_user_item[user_id]
# Reshape the user_row to have 1 row per user
user_items = np.reshape(user_row, (1, -1))

# Now you can call the recommend function
recommended = model.recommend(user_id, user_items, N=10, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab
rec_tab

,sku_id,conf,Sku
0,44289.0,0.425989,S60GJN4
1,32693.0,0.393835,CF32570
2,33245.0,0.378972,CF48091
3,34621.0,0.355968,CF73760
4,48290.0,0.348539,W000108
5,44097.0,0.320057,S60AZG9
6,43124.0,0.313102,M353442
7,29395.0,0.312875,CE52895
8,43137.0,0.285442,M353694
9,27732.0,0.284276,CE23711


In [33]:
#Get similar items
item_id = 7
n_similar = 3
similar = model.similar_items(item_id, n_similar)
print(similar)

(array([    7, 45067,  1553], dtype=int32), array([1.        , 0.9996397 , 0.99956155], dtype=float32))
